# Load data and data analysis

In [ ]:
# TODO
# TODO: maybe load the dataset as follows from huggingface
# https://huggingface.co/docs/datasets/v1.9.0/loading_datasets.html
from datasets import load_dataset
dataset = load_dataset('squad')

# Setup generative model (open/closed model)

In [ ]:
from transformers import AutoTokenizer, OPTForCausalLM, pipeline
import torch

class GenerativeModel:
    def __init__(self, max_answer_length) -> None:
        self.generator = pipeline('text-generation', model="facebook/opt-1.3b")
        self.model = OPTForCausalLM.from_pretrained("facebook/opt-1.3b")
        self.tokenizer = AutoTokenizer.from_pretrained("facebook/opt-1.3b")
        self.tokenizer.padding_side = "left"   # so that the text will continue as without padding
        self.max_answer_length = max_answer_length

    def get_open_model_answer(self, question, context, use_pipeline=False):
        prompt = f"CONTEXT:\n{context}\nQUESTION:\n{question}"
        # generate answer
        answer = self._generate_answer(prompt, use_pipeline)
        # remove prompt from generated text
        answer = answer.removeprefix(prompt)
        return answer

    def get_closed_model_answer(self, question, use_pipeline=False):
        prompt = question
        # generate answer
        answer = self._generate_answer(prompt, use_pipeline)
        # remove prompt from generated text
        answer = answer.removeprefix(prompt)
        return answer

    @torch.jit.script
    def get_open_batch_answers(self, questions, contexts, use_pipeline=False):
        assert len(questions) == len(contexts), "questions and contexts should have the same length"
        prompts = [f"CONTEXT:\n{contexts[i]}\nQUESTION:\n{questions[i]}" for i in range(len(questions))]
        # generate answers
        answers = self._generate_batch_answers(prompts, use_pipeline)
        # remove prompts from generated text
        answers = [answers[i].removeprefix(prompts[i]) for i in range(len(prompts))]
        return answers

    # https://github.com/huggingface/transformers/issues/10704
    @torch.jit.script
    def get_closed_batch_answers(self, questions, use_pipeline=False):
        prompts = questions
        # generate answers
        answers = self._generate_batch_answers(prompts, use_pipeline)
        # remove prompts from generated text
        answers = [answers[i].removeprefix(prompts[i]) for i in range(len(prompts))]
        return answers
    
    def _generate_answer(self, prompt, use_pipeline):
        if use_pipeline:
            answer = self.generator(prompt, max_new_tokens=self.max_answer_length)[0]['generated_text']
        else:
            inputs = self.tokenizer(prompt, return_tensors="pt")
            generate_ids = self.model.generate(inputs.input_ids, max_new_tokens=self.max_answer_length)
            answer = self.tokenizer.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]
        return answer

    def _generate_batch_answers(self, prompts, use_pipeline):
        if use_pipeline:
            answers = [self.generator(prompt, max_new_tokens=self.max_answer_length)[0]['generated_text'] for prompt in prompts]
        else:
            inputs = self.tokenizer(prompts, return_tensors="pt", padding=True)  # padding, so that all prompts have same length for computing it as a batch
            generate_ids = self.model.generate(inputs.input_ids, max_new_tokens=self.max_answer_length)
            answers = self.tokenizer.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)
        return answers

In [ ]:
generative_model = GenerativeModel(max_answer_length=42)

In [ ]:
# test for single question
single_context = """Super Bowl 50 was an American football game to determine the champion of the National Football League (NFL) for the 2015 season. The American Football Conference (AFC) champion Denver Broncos defeated the National Football Conference (NFC) champion Carolina Panthers 24–10 to earn their third Super Bowl title. The game was played on February 7, 2016, at Levi's Stadium in the San Francisco Bay Area at Santa Clara, California. As this was the 50th Super Bowl, the league emphasized the "golden anniversary" with various gold-themed initiatives, as well as temporarily suspending the tradition of naming each Super Bowl game with Roman numerals (under which the game would have been known as "Super Bowl L"), so that the logo could prominently feature the Arabic numerals 50."""
single_question = "Which NFL team represented the AFC at Super Bowl 50?"
single_correct_answers = ['Denver Broncos', 'Denver Broncos', 'Denver Broncos']

print(f"Context: {single_context}\nQuestion: {single_question}\nCorrect answer: {single_correct_answers}")
print("Closed generative Model:")
print(generative_model.get_closed_model_answer(single_question))
print("Open generative Model:")
print(generative_model.get_open_model_answer(single_question, single_context))

# Evaluation

In [ ]:
import nltk
from nltk.translate.bleu_score import sentence_bleu
from nltk.tokenize import word_tokenize
import numpy as np
from transformers import pipeline
nltk.download('punkt')

In [ ]:
class ExtractiveModel:
  def __init__(self) -> None:
    model_name = "deepset/roberta-base-squad2"
    self.pipeline = pipeline('question-answering', model=model_name, tokenizer=model_name)

  def get_extractive_batch_answer(self, question, context):
    answer = self.pipeline(question=question, context=context)["answer"]
    return answer

  def get_extractive_batch_answers(self, questions, contexts):
    answers = [self.pipeline(question=q, context=c)["answer"] for (q,c) in zip(questions, contexts)]
    return answers

In [ ]:
extractive_model = ExtractiveModel()

In [ ]:
# test for single question
single_context = """Super Bowl 50 was an American football game to determine the champion of the National Football League (NFL) for the 2015 season. The American Football Conference (AFC) champion Denver Broncos defeated the National Football Conference (NFC) champion Carolina Panthers 24–10 to earn their third Super Bowl title. The game was played on February 7, 2016, at Levi's Stadium in the San Francisco Bay Area at Santa Clara, California. As this was the 50th Super Bowl, the league emphasized the "golden anniversary" with various gold-themed initiatives, as well as temporarily suspending the tradition of naming each Super Bowl game with Roman numerals (under which the game would have been known as "Super Bowl L"), so that the logo could prominently feature the Arabic numerals 50."""
single_question = "Which NFL team represented the AFC at Super Bowl 50?"
single_correct_answers = ['Denver Broncos', 'Denver Broncos', 'Denver Broncos']

print(f"Context: {single_context}\nQuestion: {single_question}\nCorrect answer: {single_correct_answers}")
print("Extractive Model:")
single_answer = extractive_model.get_extractive_batch_answer(single_question, single_context)
print(single_answer)

## Compute the answers from the different models

In [14]:
# TEMP
from datasets import load_dataset
test_set = load_dataset('squad', split='validation[:100]')  # get first n entries from the test set
test_contexts = test_set['context']  # list of strings
test_questions = test_set['question']  # list of strings
test_answers = [d['text'] for d in test_set['answers']]  # list of lists of answers
#TEMP end

Found cached dataset squad (/Users/stephan/.cache/huggingface/datasets/squad/plain_text/1.0.0/d6ec3ceb99ca480ce37cdd35555d6cb2511d223b9150cce08a837ef62ffea453)


In [15]:
%%time
closed_answers = generative_model.get_closed_batch_answers(test_questions)

CPU times: user 3min 29s, sys: 3min 20s, total: 6min 49s
Wall time: 50.6 s


In [16]:
%%time
open_answers = generative_model.get_open_batch_answers(test_questions, test_contexts)

CPU times: user 11min 5s, sys: 14min 29s, total: 25min 34s
Wall time: 4min 32s


In [17]:
%%time
extractive_answers = extractive_model.get_extractive_batch_answers(test_questions, test_contexts)

CPU times: user 41.3 s, sys: 25.8 s, total: 1min 7s
Wall time: 7.72 s


## Compute the scores for the answers

In [18]:
# NOTE: BLEU cannot be used, since the sentences are often very short, and BLEU uses 4-grams, so if the answer has less than 4 words, it gets a BLEU Score of 0.0

def compute_word_recall(correct_answer_, model_answer_):
  correct_answer_words = word_tokenize(correct_answer_)
  model_answer_words = set(word_tokenize(model_answer_))
  relevant_retrieved_words = np.sum([(corr_word in model_answer_words) for corr_word in correct_answer_words])
  all_relevant_words = len(correct_answer_words)
  return relevant_retrieved_words / all_relevant_words

def compute_word_recall_multiple_answers(correct_answers_single_question_, model_answer_):
  # return the best score, since the different answers are often synonyms and if one is taken, this is fine
  return np.max([compute_word_recall(correct_answer, model_answer_) for correct_answer in correct_answers_single_question_])

def compute_word_recall_average(correct_answers_, model_answers_):
  all_recalls = [compute_word_recall_multiple_answers(correct_answer, model_answer)for (correct_answer, model_answer) in zip(correct_answers_, model_answers_)]
  return np.mean(all_recalls)

In [19]:
closed_recall = compute_word_recall_average(test_answers, closed_answers)
closed_recall

0.2517857142857143

In [20]:
open_recall = compute_word_recall_average(test_answers, open_answers)
open_recall

0.8441666666666666

In [21]:
extractive_recall = compute_word_recall_average(test_answers, extractive_answers)
extractive_recall

0.9816666666666666

## Plots